In [18]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
import torchvision

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        for param in self.model.parameters():
            param.requires_grad = False

        self.linear = nn.Sequential(
            nn.Linear(1000, 256),
            nn.Sigmoid(),
            nn.Dropout(0.1),
            nn.Linear(256, 256),
            nn.Sigmoid(),
            nn.Dropout(0.1),
            nn.Linear(256, 4),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.linear(self.model(x))


In [19]:
import torchvision.transforms as tt

train_dir = 'images/train/'
test_dir = 'images/test/'
train_dataset = torchvision.datasets.ImageFolder(
    train_dir,
    transform=tt.Compose([
        tt.RandomResizedCrop(224),
        tt.RandomHorizontalFlip(),
        tt.ToTensor(),
        tt.Normalize((0.485*255, 0.456*255, 0.406*255), 
                     (0.229*255, 0.224*255, 0.225*255))
    ])
)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=10, 
    shuffle=True, 
    num_workers=2, 
    pin_memory=True
)

test_dataset = torchvision.datasets.ImageFolder(
    test_dir,
    transform=tt.Compose([
        tt.Resize((224, 224)),
        tt.ToTensor(),
        tt.Normalize((0.485*255, 0.456*255, 0.406*255),
                     (0.229*255, 0.224*255, 0.225*255))
    ])
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=True,
    pin_memory=True
)

In [21]:
model = Classifier()
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.0001)
for epoch in range(30):
    print(f'Epoch {epoch}')
    model.train()
    train_accuracy = torch.tensor(0.)
    for X, y in train_dataloader:
        optimizer.zero_grad()
        output = model(X)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            train_accuracy += (torch.argmax(output, dim=1) == y).sum()
    print(f'Training accuracy: {train_accuracy/len(train_dataloader.dataset):.3f}')

Epoch 0
Training accuracy: 0.263
Epoch 1
Training accuracy: 0.303
Epoch 2
Training accuracy: 0.314
Epoch 3
Training accuracy: 0.311
Epoch 4
Training accuracy: 0.316
Epoch 5
Training accuracy: 0.324
Epoch 6
Training accuracy: 0.352
Epoch 7
Training accuracy: 0.364
Epoch 8
Training accuracy: 0.405
Epoch 9
Training accuracy: 0.442
Epoch 10
Training accuracy: 0.454
Epoch 11
Training accuracy: 0.459
Epoch 12
Training accuracy: 0.488
Epoch 13
Training accuracy: 0.491
Epoch 14
Training accuracy: 0.488
Epoch 15
Training accuracy: 0.518
Epoch 16
Training accuracy: 0.553
Epoch 17
Training accuracy: 0.557
Epoch 18
Training accuracy: 0.598
Epoch 19
Training accuracy: 0.615
Epoch 20
Training accuracy: 0.614
Epoch 21
Training accuracy: 0.644
Epoch 22
Training accuracy: 0.681
Epoch 23
Training accuracy: 0.684
Epoch 24
Training accuracy: 0.722
Epoch 25
Training accuracy: 0.757
Epoch 26
Training accuracy: 0.765
Epoch 27
Training accuracy: 0.806
Epoch 28
Training accuracy: 0.828
Epoch 29
Training accura

In [33]:
test_accuracy = torch.tensor(0.)
for X, y in test_dataloader:
    output = model(X)
    test_accuracy += (torch.argmax(output, dim=1) == y).sum()
print(f'Training accuracy: {test_accuracy/len(test_dataloader.dataset):.3f}')

Test accuracy: 0.663
